<h3> Grouping data

<h4> Group by Mechanics

In [5]:
import psycopg2
conn  = psycopg2.connect(dbname = 'postgres', user = 'postgres', password = 'durham17')
conn.close()

In [5]:
import pandas as pd 
import numpy as np

df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
     'key2' : ['one', 'two', 'one', 'two', 'one'],
     'data1' : np.random.randn(5),
     'data2' : np.random.randn(5)})

In [6]:
grouped = df['data1'].groupby(df['key1']) #identify the column on which we will perform the grouping 
#                                            this will usuallly be a colummn within the same dataframe
#                                            therefore, a simple 'key1' also makes up for it

In [7]:
grouped #nothing yet calculated, the object is simply now a Grouped object
        #allowing us to perform any operation on the grouped data like sum, avg etc on the group 

In [8]:
grouped.sum()

key1
a    1.904083
b   -1.208443
Name: data1, dtype: float64

In [9]:
new_grouped = df['data1'].groupby([df['key1'], df['key2']]).sum()
new_grouped #usually you want to perform the grouping in terms of a column 
            #alrady existing in the dataframe, hence you just pass the labels as an argument 
            #in the group by function

key1  key2
a     one     1.171297
      two     0.732787
b     one    -0.876256
      two    -0.332187
Name: data1, dtype: float64

In [10]:
new_grouped.unstack() #since we have multiple layers of indices we can play around by unstacking 

key2,one,two
key1,,
a,1.171297,0.732787
b,-0.876256,-0.332187


In [11]:
df.groupby(['key1', 'key2']).size() #count the elements within each group

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [9]:
for (k1,k2), group in df.groupby(['key1', 'key2']):
    print(k1,k2)
    print(group)
#groupby object is iterable, therefore we can parse it with a for 
#getting the each separate key and its chunk of data

a one
  key1 key2     data1     data2
0    a  one  1.146703  0.041228
4    a  one -0.064465  1.728981
a two
  key1 key2     data1   data2
1    a  two -0.549787  0.9125
b one
  key1 key2     data1    data2
2    b  one -1.294528 -0.23877
b two
  key1 key2     data1     data2
3    b  two -2.183422 -1.643006


In [10]:
pieces = list(df.groupby('key1')) #since groupby is iterable we cast the object into list like 
                                  #objects i.e. lists, tuples, dicts 

In [11]:
pieces

[('a',
    key1 key2     data1     data2
  0    a  one  1.146703  0.041228
  1    a  two -0.549787  0.912500
  4    a  one -0.064465  1.728981),
 ('b',
    key1 key2     data1     data2
  2    b  one -1.294528 -0.238770
  3    b  two -2.183422 -1.643006)]

In [12]:
pieces = dict(pieces)

In [13]:
pieces

{'a':   key1 key2     data1     data2
 0    a  one  1.146703  0.041228
 1    a  two -0.549787  0.912500
 4    a  one -0.064465  1.728981,
 'b':   key1 key2     data1     data2
 2    b  one -1.294528 -0.238770
 3    b  two -2.183422 -1.643006}

In [14]:
grouped = df.groupby(df.dtypes, axis = 1) #remember that groupby groups by default on 0 axis 
                                          #that said, along the rows

for data_types, data in grouped: #group data by their data types
    print (data_types)
    print(data)

float64
      data1     data2
0  1.146703  0.041228
1 -0.549787  0.912500
2 -1.294528 -0.238770
3 -2.183422 -1.643006
4 -0.064465  1.728981
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [15]:
for data_types, data in grouped: #group data by their data types
    print (data_types)
    print(data)

float64
      data1     data2
0  1.146703  0.041228
1 -0.549787  0.912500
2 -1.294528 -0.238770
3 -2.183422 -1.643006
4 -0.064465  1.728981
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [16]:
df.groupby('key1').sum()

,data1,data2
key1,,
a,0.53245,2.682709
b,-3.47795,-1.881776


In [17]:
df.groupby('key1')['data1'].sum() #brings out a series

key1
a    0.53245
b   -3.47795
Name: data1, dtype: float64

In [18]:
    df.groupby('key1')[['data1']].sum() #brings out a dataframe

,data1
key1,
a,0.53245
b,-3.47795


<h4> Data Aggregation 

<p>  Data aggregation functions are simply the functions that act on multidimensional 
    arrays to return a scalar value. Functions like these inclde sum(), average(), count(), 
    min(), max(), median(), var(), std(), etc...
   

<p> One can define his own aggregation function by defining the function itself and then passing its process to the data via the .agg() method e.g.:
    
    def peak_to_peak(arr):
        return arr.max() - arr.min()
    grouped.agg(peak_to_peak) 
    
in case the function to be used is already a built-in Python function then we can pass it through a 
string e.g.:
    
    grouped.agg('sum')


<p> Once we group by a dataframe by two or more column element (this will result in high level layered
    index). We can aggregate by particular functions (built-in or user defined). Just pass the    
    functions as list like object within .agg().
    
<p> Here we can also give our own defined names for each of these functions, these will then appear as 
    the names of the columns in the new dataframe. Syntax is implemented as 2-tuple e.g.:
    
        grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

<p> In case we want to return back the dataframe with a reset index after performing the grouping 
    we can add an optional argument the as_index = False (or call the dataframe.reset_index in the 
    next line)

<h4> General split - apply -combine

In [19]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
        'data2': np.random.randn(1000)}) 

In [20]:
frame

,data1,data2
0,-0.020104,1.045619
1,-2.318474,0.954577
2,-0.320897,1.422115
3,1.156208,-0.330006
4,0.282310,-0.838154
...,...,...
995,-1.165283,-0.214872
996,-0.352671,-0.957109
997,-0.894165,-0.391920
998,-0.337819,0.144592


In [21]:
quartiles = pd.cut(frame.data1, 4) #let's break the data into 4 buckets of equal 
                                   #length 

In [22]:
quartiles[:10]

0    (-0.182, 1.605]
1    (-3.765, -1.97]
2    (-1.97, -0.182]
3    (-0.182, 1.605]
4    (-0.182, 1.605]
5    (-0.182, 1.605]
6    (-0.182, 1.605]
7    (-0.182, 1.605]
8    (-1.97, -0.182]
9    (-1.97, -0.182]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.765, -1.97] < (-1.97, -0.182] < (-0.182, 1.605] < (1.605, 3.392]]

<p> Therefore, cut returns a categorical object that can in turn be passed in a groupby method
    to come up with an elementary analysis.

In [23]:
def get_stats(group):
       return {'min': group.min(), 'max': group.max(),
       'count': group.count(), 'mean': group.mean()} #create a dict that will return a frame with 
                                                     #included values/results of functions 

In [24]:
grouped = frame.data2.groupby(quartiles).apply(get_stats)

In [25]:
grouped #we would like to unstack this so that it will look closer to a dataframe

data1                 
(-3.765, -1.97]  min       -1.290351
                 max        1.832438
                 count     28.000000
                 mean       0.278480
(-1.97, -0.182]  min       -3.097640
                 max        3.085691
                 count    396.000000
                 mean      -0.046310
(-0.182, 1.605]  min       -2.760482
                 max        3.487996
                 count    513.000000
                 mean      -0.012373
(1.605, 3.392]   min       -2.288093
                 max        1.550225
                 count     63.000000
                 mean      -0.061613
Name: data2, dtype: float64

In [26]:
grouped.unstack() #this will add a higher column layer 

,min,max,count,mean
data1,,,,
"(-3.765, -1.97]",-1.290351,1.832438,28.0,0.278480
"(-1.97, -0.182]",-3.097640,3.085691,396.0,-0.046310
"(-0.182, 1.605]",-2.760482,3.487996,513.0,-0.012373
"(1.605, 3.392]",-2.288093,1.550225,63.0,-0.061613


In [27]:
grouping = pd.qcut(frame.data1, 10, labels = False) #take out the labels to just show the quartiles

In [28]:
grouped = frame.data1.groupby(grouping).apply(get_stats)

In [29]:
grouped.unstack() #unstack the Series like object and voila

,min,max,count,mean
data1,,,,
0,-3.757417,-1.281580,100.0,-1.812180
1,-1.279988,-0.819359,100.0,-1.044174
2,-0.818913,-0.524604,100.0,-0.659369
3,-0.522969,-0.249779,100.0,-0.370770
4,-0.243360,0.007964,100.0,-0.121458
5,0.009090,0.272704,100.0,0.126368
6,0.273398,0.543485,100.0,0.399724
7,0.544162,0.896555,100.0,0.730134
8,0.903433,1.348839,100.0,1.098143


<p> Let's come back to the context of missing values, in general we will either drop these values 
    (usually this is the case where our analysis is not affected by these values) or fill these 
    missing data with values coming from the frame (usually the mean, median or most frequent value 
    in case of a categorical variable).

In [30]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan #generate missing values in the Series

In [31]:
s

0         NaN
1   -0.303752
2         NaN
3    0.826533
4         NaN
5    0.768268
dtype: float64

In [32]:
s.fillna(s.mean())

0    0.430350
1   -0.303752
2    0.430350
3    0.826533
4    0.430350
5    0.768268
dtype: float64

In [33]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
        'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4

In [34]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [35]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -2.722728
New York     -0.759869
Vermont      -0.950695
Florida       0.341104
Oregon       -0.749584
Nevada       -1.674508
California   -0.531549
Idaho        -1.484340
dtype: float64

In [36]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [37]:
data

Ohio         -2.722728
New York     -0.759869
Vermont            NaN
Florida       0.341104
Oregon       -0.749584
Nevada             NaN
California   -0.531549
Idaho              NaN
dtype: float64

In [38]:
fill_mean = lambda g: g.fillna(g.mean())

In [39]:
data.groupby(group_key).apply(fill_mean) #we just replace the missing value for west/east city 
                                         #with the mean from for west/east

Ohio         -2.722728
New York     -0.759869
Vermont      -1.047164
Florida       0.341104
Oregon       -0.749584
Nevada       -0.640566
California   -0.531549
Idaho        -0.640566
dtype: float64

In [40]:
data.groupby(group_key).mean()

East   -1.047164
West   -0.640566
dtype: float64

In [43]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name]) #take advantage of the internal
                                                    #attribute g.name of a groupby object?
data.groupby(group_key).apply(fill_func)

Ohio         -2.722728
New York     -0.759869
Vermont       0.500000
Florida       0.341104
Oregon       -0.749584
Nevada       -1.000000
California   -0.531549
Idaho        -1.000000
dtype: float64